# `tifuun-sensitivity` tutorial. 

In this tutorial for `tifuun-sensitivity`, we will discuss how to use the program. It is divided into three sections:
- Defining input structures and parameters.
- Running a calculation.
- Understanding the output format.

First, we discuss the input structures. We will define the telescope and instrument. Then, we will discuss an important input: the radiative transfer cascade. Lastly, we discuss some other input parameters that pertain to details unrelated to the telescope, instrument, or cascade, which are important for calculating some output quantities. 

After that, we show quickly how to use the `tifuun-sensitivity` package and how to call the relevant functions.

Finally, we discuss the output given by `tifuun-sensitivity`.

Throughout this tutorial, we will use parameters of DESHIMA 2.0.

## Part 1: input to calculation

The telescope and instrument are defined using dictionaries. These dictionaries have the same structure as the input dictionaries for the `gateau` package (the end-to-end simulator for TIFUUN). However, the required parameters only form a subset of the `gateau` input since the sensitivity calculation is significantly more simple than the full simulation. We will now describe each dictionary in detail.

### Telescope dictionary

The important parameters for the telescope are: diameter, RMS surface roughness, aperture efficiency, and elevation. These should be assigned to a dictionary, where the field names should be as in the following example:

In [7]:
import numpy as np
import matplotlib.pyplot as plt

from tifuun_sensitivity.simulator import calculator

diameter = 10    # Diameter, units: m
s_rms    = 42e-6 # RMS surface roughness, units: m
eta_ap   = 0.7   # Aperture efficiency at zero frequency (so not including Ruze efficiency), units: none
EL0      = 60    # Elevation angle of telescope with respect to the horizon, units: degrees

telescope = {
    "D_tel"   : diameter,
    "s_rms"   : s_rms,
    "eta_ap"  : eta_ap,
    "EL0"     : EL0,
}

Note that we also put the imports here for `numpy`, `matplotlib`, and `tifuun-sensitivity`. We will need these later on.

### Instrument dictionary

The instrument is quite important, as this sets the response curves for the filterbank. Currently, these can be Lorentzians, or measured shapes loaded from a file. 

If a Lorentzian filterbank is required, the resolving power and central channel frequencies should be passed. Additionally, the in-band fraction should be given. This is a number specifying how much of the filter couples to line emission and continuum emission. This number is usually 0.5, meaning that continuum emission loads a filter shape more by about a factor 2. Since it is really not a necessity of `gateau` to calculate different coupling efficiencies for line and continuum, this factor is unique for `tifuun-sensitivity`. Another, optional, parameter is the KID excess noise factor. If this is not given, it is set to 1 for the calculation. This factor is also unique to `tifuun-sensitivity`.

An instrument dictionary with a Lorentzian filterbank is specified as follows:

In [8]:
f0 = 220
nf = 350
R = 500

f_ch = f0 * (1 + 1/R)**np.arange(nf) 

instrument = {
    "f_ch"    : f_ch, # Channel frequencies in GHz
    "R"       : R,    # Resolving power
    "eta_IBF" : 0.5, # in-band fraction
    "KID_excess_noise_factor" : 1.1
}

For a custom filterbank, the `"f_ch"` field should contain a `numpy` array with the center frequencies of each filter. The `"R"` field should contain a `numpy` array with shape `(2,N)`, with the first column containing the frequencies at which the filterbank has been measured in GHz, and the second column contains the filter shapes. Note that, if the measured filters have not been normalised (and thus contain the transmission coefficient as well), the final efficiency in the cascade (which couples to the filterbank) should be set at 1. We will come back to this when discussing the cascade in the next section.

Note that, when working from Lorentzian filterbanks, `tifuun-sensitivity` will calculate the frequency points at which to evaluate the filterbank for you. That's why this does not need to be passed when generating the filterbank from a single "R" value.

### Cascade List

The cascade list is a list containing dictionaries. Each of these dictionaries defines a radiative transfer stage of the cascade, where the incoming radiation is attenuated by a specific efficiency term pertaining to the stage, and radiation is added due to the emissivity of the stage. In formula form:
$$P_{\nu,\mathrm{out}} = \eta_\mathrm{stage} P_{\nu,\mathrm{in}} + (1 - \eta_\mathrm{stage}) P_{\nu,\mathrm{stage}},$$
where $P_{\nu,\mathrm{out}}$ is the power spectral density of the outgoing radiation, $\eta_\mathrm{stage}$ the efficiency term associated with this stage, $P_{\nu,\mathrm{in}}$ the power spectral density of the incoming radiation, and $P_{\nu,\mathrm{stage}}$ the single-moded Johnson-Nyquist power spectral density of the radiation emitted by the stage itself, which we will often refer to as the 'parasitic' source.

A dictionary for a single stage can be divided into two types: a **reflective stage**, where light solely reflects off the stage as is the case for, for example, spillover losses or Ohmic losses, or a **refractive stage**, which is the case for cryostat windows and lenses. Note that refractive stages could also contain reflective stages if they are not AR coated. We will start by discussing reflective stages.

#### Reflective cascade stages
These require two dictionary fields: one containing the efficiency of the stage, named "eta_coup", and one containing the physical temperature of the parasitic source, named "T_parasitic". Let's say we would like to include spillover losses on the primary reflector of the telescope. Let's assume the spillover efficiency is 0.99. Also, the part of the beam that spills over will couple to the ground, which has a physical temperature of 273 K (for example). The dictionary would look like this:

In [9]:
cascade_list = []

primary_spillover = {
    "eta_coup"     : 0.99,
    "T_parasitic"  : 273
}

cascade_list.append(primary_spillover)

Now, say this primary reflector is made out of aluminium. Therefore, Ohmic losses should be included. This can be done by defining a new stage.

In [10]:
primary_Ohmic = {
    "eta_coup"     : "Ohmic-Al",
    "T_parasitic"  : 273
}

cascade_list.append(primary_Ohmic)

By passing "Ohmic-Al" as "eta_coup", it tells `tifuun-sensitivity` to use the efficiency of aluminium. This is calculated from measured values at 850 GHz, and converted to the range used by the filterbank. 

The next stage would be the secondary mirror. We will assume a spillover efficiency of 0.9 for this. However, the part of the beam that spills over does not couple to 273 K now, but rather directly to the atmosphere. The atmosphere power spectral density has a rather complex shape that cannot be parameterised by a single temperature and hence needs a special way of incorporating. This can be done in the following way:

In [11]:
secondary_spillover = {
    "eta_coup"     : 0.9,
    "T_parasitic"  : "atmosphere"
}

cascade_list.append(secondary_spillover)

This tells `tifuun-sensitivity` that it should use the atmosphere power spectral density in this stage as the parasitic source. This is calculated already in the beginning of the cascade using the physical temperature of the atmosphere and the precipitable water vapor (pwv) (see next subsectuion on inputs) and the `ATM` model to calculate transmission of the atmosphere as function of frequency and pwv.

### Other input parameters

Aside from the input dictionaries, `tifuun-sensitivity` requires a couple of other input parameters. These are passed separately, because they do not depend on the telescope or instrument, but rather on the atmosphere and observation length or requirements. the extra parameters are:

In [12]:
PWV = 0.5 # Precipitable water vapor, units: mm
Tb_cmb = 2.725 # CMB temperature, units: K
Tp_atm = 273 # Physical temperature of atmosphere, units: K
snr = 5 # Signal-to-noise required for a line to be detectable. Used for calculating MDLF. Units: none
obs_hours = 1 # Number of hours spent observing.Used for calculating MDLF.
on_source_fraction = 0.4 * 0.9 # How much of the scanning time is actually spent on-source. This also reflects chopping.
on_off = True # Whether or not to perform sky chopping

## Running the calculation

After defining all input parameters, it is really simple to start the calculation: